In [ ]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import equistore
from equistore import Labels, TensorBlock, TensorMap

# from rascaline.utils import clebsch_gordan
import clebsch_gordan

In [ ]:
frames = [ase.io.read("frame.xyz")]

lmax = 5
lambdas = np.array([0, 2])
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": lmax,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

n_body = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    lambda_cut=lmax * 2,
    species_neighbors=[1, 8, 6],
)
n_body

In [ ]:
selected_samples = None
species_neighbors = [1, 8, 6]

calculator = rascaline.SphericalExpansion(**rascal_hypers)
nu1 = calculator.compute(frames, selected_samples=selected_samples)

# Move the "species_neighbor" key to the properties. If species_neighbors is
# passed as a list of int, sparsity can be created in the properties for
# these species.
if species_neighbors is None:
    keys_to_move = "species_neighbor"
else:
    keys_to_move = Labels(
        names=["species_neighbor"],
        values=np.array(species_neighbors).reshape(-1, 1),
    )
nu1 = nu1.keys_to_properties(keys_to_move=keys_to_move)
nu1 = clebsch_gordan._add_nu_sigma_to_key_names(nu1)